# Instalando a SDK do Facebook para Python

Para os exemplos desse notebook iremos utilizar a biblioteca do Facebook para Python.

Mais informações podem ser conferidas na documentação: https://facebook-sdk.readthedocs.io/en/latest/index.html

In [3]:
!pip install facebook-sdk

In [1]:
import facebook as fb

# Autenticando o APP

Para coletarmos informações relacionadas ao usuário vamos usar o token de usuário gerado na GRAPH API

In [2]:
token= 'insira o seu token aqui'

In [3]:
graph = fb.GraphAPI(token)

Agora podemos verificar se somos capazes de acessar as informações do nosso próprio usuário.

In [4]:
profile = graph.get_object("me")

In [5]:
profile

{'name': 'Júlio C. Oliveira', 'id': '2221772204572402'}

In [113]:
permissions = graph.get_permissions(user_id=2221772204572402)
print('public_profile' in permissions)

True


In [114]:
permissions

{'manage_pages',
 'pages_messaging',
 'pages_show_list',
 'public_profile',
 'publish_pages',
 'publish_video',
 'user_events',
 'user_friends',
 'user_hometown',
 'user_likes',
 'user_photos',
 'user_posts',
 'user_status'}

# Dados do usuário

A postagem de conteúdos no feed é desabilitada por padrão para usuários

In [8]:
graph.put_object("me", "feed", message="I am writing on my wall!")

GraphAPIError: (#200) If posting to a group, requires app being installed in the group, and \
          either publish_to_groups permission with user token, or both manage_pages \
          and publish_pages permission with page token; If posting to a page, \
          requires both manage_pages and publish_pages as an admin with \
          sufficient administrative permission

A Graph API permite recuperar dados públicos de usuários e páginas, mas é necessário solicitar avaliação do Facebook para ter essa permissão no app.

In [9]:
user = "BillGates"

In [10]:
profile = graph.get_object(user)

GraphAPIError: (#10) To use 'Page Public Content Access', your use of this endpoint must be reviewed and approved by Facebook. To submit this 'Page Public Content Access' feature for review please read our documentation on reviewable features: https://developers.facebook.com/docs/apps/review.

# Paginando os resultados

A quantidade de dados retornados em uma chamada é limitada, sendo assim para recuperarmos todas as informações em chamadas maiores utilizamos a paginação.

In [13]:
profile = graph.get_object("me")

O método abaixo retorna todos os posts do usuário.

In [15]:
posts = graph.get_connections(profile["id"], "posts")

In [16]:
posts

{'data': [{'message': 'Uma ótima dica para quem quer aprender um pouco mais sobre o mundo de tecnologia, são os cursos da Alura Cursos Online.  \n\nAproveita essa oportunidade, confere esse  curso de Machine Learning no Marketing Digital que tive o prazer de construir com essa galera.',
   'created_time': '2019-05-09T23:46:30+0000',
   'id': '2221772204572402_2213959718686984'},
  {'message': '"...a educação das crianças não se limita a memorizar informações e fatos. O conhecimento existe em um contexto, numa abordagem que, necessariamente, envolve o desenvolvimento emocional, social, intelectual, moral e físico do aluno."',
   'created_time': '2019-01-08T10:16:24+0000',
   'id': '2221772204572402_2037517616331196'},
  {'message': 'Sera se agora temos pote de bitcoin no final do arco-íris? #home #sp',
   'created_time': '2018-11-24T21:38:09+0000',
   'id': '2221772204572402_1975631722519786'},
  {'message': 'A culpa é dos quadrúpedes, espera, é dos anfíbios, digo é da direita, me engan

Olhando a chave "pagingin" e depois "next" podemos notar uma url, que podemos abrir com o próprio navegador e verificar que ela contém a continuação da chamada anterior.

In [17]:
posts["paging"]["next"]

'https://graph.facebook.com/v3.3/2221772204572402/posts?access_token=EAAFlbR4iAoMBACd5dBUMvGP0tZChHV8GUbJ2P9nQJ4p0nYVHzkvQiBjAYNbghUPHKEpjNoCadsx4Rpp3VuTWDIQMYfI4hHaNthbZChNpn7v5Vh6nFJSCrMAMu5V2vZBZA7ZAnlLJVQQRsiMY7qQSZCXTDdcZBoX0nqHQJFsZAloWmdwdvf27hZBsEnmGtwVBZCYmDYZC9RBlcVdGwZDZD&limit=25&until=1508364009&__paging_token=enc_AdCEOq4T0f7EZCNfa4uo9QsCij1wfxi3gq5Ye33RaEdvrrQMfVsc9lZBlQApKlMKrDi3S7yq3k3cFSDcSzRLXJ5SoIScq57QZCZAxSziZAjI3oecsbgZDZD'

Vamos utilizar a biblioteca requests para fazer chamadas http, do mesmo modo que fizemos com o nosso navegador para abrir a url anterior.

In [ ]:
import requests

In [19]:
posts = requests.get(posts["paging"]["next"])

In [20]:
posts.json()

{'data': [{'created_time': '2017-10-11T10:11:39+0000',
   'id': '2221772204572402_1499321606817469'},
  {'created_time': '2017-09-14T22:41:51+0000',
   'id': '2221772204572402_1477188119030818'},
  {'created_time': '2017-09-13T23:40:30+0000',
   'id': '2221772204572402_1476371459112484'},
  {'message': 'Nosso cotidiano! #muchmore #fiat #jeep \n\n',
   'created_time': '2017-09-07T14:58:59+0000',
   'id': '2221772204572402_1471214706294826'},
  {'created_time': '2017-08-22T23:13:32+0000',
   'id': '2221772204572402_1458156344267329'},
  {'created_time': '2017-08-11T21:58:54+0000',
   'id': '2221772204572402_1448779951871635'},
  {'message': 'Sim, \n\nestamos no Catraca Livre! :)\n\n',
   'created_time': '2017-05-16T19:23:13+0000',
   'id': '2221772204572402_1362480753834889'},
  {'message': '"Há alguns meses vinha sentindo a necessidade (e vontade) de aprender um pouco mais sobre o Excel. Tive a oportunidade de fazer o curso com o Júlio e fiquei muito feliz com o resultado, especialmente

Caso tenhamos muitos posts em nossa conta, é interessante limitar a quantidade de paginação para evitar estourar o limite de API.

In [22]:
pagination = 3

Agora podemos fazer um loop para ler os resultados das próximas páginas e adicionar os posts a lista que criamos.

In [27]:
posts = []
page = graph.get_connections(profile["id"], "posts")

posts.append(page)

for number in range(pagination):
    page = requests.get(page["paging"]["next"]).json()
    posts.append(page)

In [28]:
len(posts)

4

In [29]:
posts[0]

{'data': [{'message': 'Uma ótima dica para quem quer aprender um pouco mais sobre o mundo de tecnologia, são os cursos da Alura Cursos Online.  \n\nAproveita essa oportunidade, confere esse  curso de Machine Learning no Marketing Digital que tive o prazer de construir com essa galera.',
   'created_time': '2019-05-09T23:46:30+0000',
   'id': '2221772204572402_2213959718686984'},
  {'message': '"...a educação das crianças não se limita a memorizar informações e fatos. O conhecimento existe em um contexto, numa abordagem que, necessariamente, envolve o desenvolvimento emocional, social, intelectual, moral e físico do aluno."',
   'created_time': '2019-01-08T10:16:24+0000',
   'id': '2221772204572402_2037517616331196'},
  {'message': 'Sera se agora temos pote de bitcoin no final do arco-íris? #home #sp',
   'created_time': '2018-11-24T21:38:09+0000',
   'id': '2221772204572402_1975631722519786'},
  {'message': 'A culpa é dos quadrúpedes, espera, é dos anfíbios, digo é da direita, me engan

# Eventos

Através da Graph API podemos acessar também os eventos que o usuário está relacionado.

In [107]:
event_ids = ['454585172012972']
events = graph.get_objects(ids=event_ids, fields='attending_count,declined_count,description')

"\nfor event in events:\n    print(event['declined_count'])\n"

In [108]:
events

{'454585172012972': {'attending_count': 124,
  'declined_count': 0,
  'description': 'ALGUMA COISA ACONTECEU.\n\nNão sabemos como, nem quando, nem onde.\n\nMas, ao longo dos ultimos 500 anos, muita coisa aconteceu com você, Brasil.\n\nPensa que está com problemas? Piorando a cada dia? Não tem mais esperança de crescer? Você não está doente, Brasil. Mas também notamos que não está bem.\n\nVocê só precisa de ajuda, Brasil. E precisamos te entender para poder te ajudar.\n\nNos conta, o que foi?\n\nAgendamos uma sessão no divã para você! Dia 13 de Julho, no Teatro Raul Cortez, em São Paulo!\n\nEstá pronto para essa sessão com a BRASA? Saiba mais e se prepare para o nosso lançamento em https://www.brasaemcasa.gobrasa.org/',
  'id': '454585172012972'}}

# Lugares

A API de pesquisa  também é liberada para uso dentro da Graph API

In [39]:
places = graph.search(type='place',
                      q="Parque Ibirapuera",
                      fields='name,location')

In [40]:
places

{'data': [{'name': 'Parque Ibirapuera',
   'location': {'city': 'São Paulo',
    'country': 'Brazil',
    'latitude': -23.58853,
    'longitude': -46.6519,
    'state': 'SP',
    'street': 'Parque Ibirapuera',
    'zip': '04002-010'},
   'id': '824746247674354'},
  {'name': 'Parque Ibirapuera Conservação',
   'location': {'city': 'São Paulo',
    'country': 'Brazil',
    'latitude': -23.583265668999,
    'longitude': -46.662964224815,
    'state': 'SP',
    'street': 'Av. República do Líbano, 331',
    'zip': '04501-000'},
   'id': '224494264231112'},
  {'name': 'Parque Viva',
   'location': {'city': 'La Guácima',
    'country': 'Costa Rica',
    'latitude': 9.9727564907748,
    'longitude': -84.249570846558,
    'street': '124',
    'zip': '20105'},
   'id': '613918685360759'},
  {'name': 'Parque Ibirapuera',
   'location': {'city': 'São Paulo',
    'country': 'Brazil',
    'latitude': -23.7306199,
    'longitude': -46.70499,
    'state': 'SP',
    'street': 'Parque Ibirapuera',
    '

In [41]:
places['data'][0]

{'name': 'Parque Ibirapuera',
 'location': {'city': 'São Paulo',
  'country': 'Brazil',
  'latitude': -23.58853,
  'longitude': -46.6519,
  'state': 'SP',
  'street': 'Parque Ibirapuera',
  'zip': '04002-010'},
 'id': '824746247674354'}

# Trabalhando com páginas

Os acessos as páginas constumam ser mais amplos do que acesso de usuários, para isso vamos utilizar o token gerado para páginas e testar comandos que são bloqueados para usuários.

In [94]:
page_token = 'insira o token da pagina aqui'

In [95]:
page_graph = fb.GraphAPI(page_token)

Assim como fizemos para usuários podemos fazer chamadas em páginas.

In [96]:
profile = page_graph.get_object("454298752041086")

In [97]:
profile

{'name': 'Aulas Python - Júlio Oliveira', 'id': '454298752041086'}

Agora podemos testar a postagem de uma mensagem no feed.

In [55]:
page_graph.put_object("454298752041086", "feed", message="Postando na minha página!")

{'id': '454298752041086_454303582040603'}

O código abaixo posta uma foto que está na máquina no feed de notícias.

In [57]:
page_graph.put_photo(image=open('mat-velloso-matvelloso-difference-between-machine-learning-and-al-if-37995538.png', 'rb'),
                message='True!')

{'id': '454307575373537', 'post_id': '454298752041086_454307575373537'}

Podemos fazer um comentário na foto que acabamos de postar

In [58]:
page_graph.put_object(parent_object='454298752041086_454307575373537', connection_name='comments',
                  message='Primeiro!')

{'id': '454307575373537_454308155373479'}

E podemos responder o comentário que fizemos.

In [59]:
page_graph.put_comment(object_id='454307575373537_454308155373479', message='haha')

{'id': '454307575373537_454309002040061'}

Agora que verificamos a possibilidade de recuperar posts e fazer comentários, podemos automatizar um fluxo para ler os comentários que não foram respondidos e responder automaticamente.

In [61]:
comments = page_graph.get_connections(id='454298752041086_454307575373537', connection_name='comments')

In [67]:
comments

{'data': [{'created_time': '2019-05-15T00:17:41+0000',
   'from': {'name': 'Aulas Python - Júlio Oliveira', 'id': '454298752041086'},
   'message': 'Primeiro!',
   'id': '454307575373537_454308155373479'}],
 'paging': {'cursors': {'before': 'MQZDZD', 'after': 'MQZDZD'}}}

In [ ]:
respondidos = []

In [80]:
comments = page_graph.get_connections(id='454298752041086_454307575373537', connection_name='comments')

for comment in comments['data']:
    if comment['id'] not in respondidos:
        page_graph.put_comment(object_id=comment['id'], 
                           message='Obrigado pelo comentário, entraremos em contato.')
        respondidos.append(comment['id'])
        print("Comentário respondido: ", comment['id'])

Comentário respondido:  454307575373537_454316182039343


Por último vamos inserir um pouco mais de inteligência no nosso código e traduzir a nossa resposta automaticamente para o idioma do comentário.

In [81]:
from textblob import TextBlobxtBlob

In [92]:
comments = page_graph.get_connections(id='454298752041086_454307575373537', connection_name='comments')

for comment in comments['data']:
    if comment['id'] not in respondidos:
        idioma = TextBlob(comment['message']).detect_language()
        
        if idioma != 'pt':
            resposta = TextBlob('Obrigado pelo comentário, entraremos em contato.').translate(to=idioma).raw
        else:
            resposta = 'Obrigado pelo comentário, entraremos em contato.'
        
        page_graph.put_comment(object_id=comment['id'], 
                           message=resposta)
        
        respondidos.append(comment['id'])
        print("Comentário respondido: ", comment['id'])

Comentário respondido:  454307575373537_454321918705436
